# Compairing the Neighborhoods of Chicago & Houston
By: Nick Adelberger

## Introduction

Chicago, on Lake Michigan in Illinois, is the 3rd largest city in the United States and is home to more than 2.7 million people. Known as "The Windy City", Chicago is an international hub for finance, culture, commerce, industry, education, technology, telecommunications, and transportation. Houston, popularly known as "The Bayou City", is set to become the 4th largest city in the United States by the second half of 2020. Houston's economy has a broad industrial base in energy, manufacturing, aeronautics, and transportation. It has become a global city, with strengths in culture, medicine and research.
    
While both cities share similarities and differences in industry, they both contain a diverse cultural population within them. To discover the diversity within each city, we will use Data to compare the neighborhoods of the two cities and determine how similar or dissimilar they are.

## Data

The Data we will be using to discover similarities and/or dissimilarities on Chicago and Houston will first come from creating a data frame containing the Neighborhoods and Boroughs within both city limits. This information was found on wikipedia through a Google search. We will also need the geographical coordinates (latitude & longitude) to each neighborhood so that we can successfully utilize the Foursquare API to pull location data on the types of venues within each neighborhood. Therefore, we will have two seperate data frames, both containing the Neighborhoods, Boroughs, Latitude and Longitude Coordinates for Chicago and Houston. 

We will use the Foursquare API location data as our main source of exploration, using what we find to gain insights into the type of venues located in each neighborhood. Foursquare is the most trusted, independent location data platform for understanding how people move through the real world. By creating a developers account, we will be able to use their platform to exract real world data on each neighborhood in Chicago and Houston. This data will include Venues, Restaurants, Hotels, Businesses, Parks, Bars, Shopping Centers, etc. 

Our final source of data will come from optaining city demographics such as age, gender, race, and income. This will be used to further explore the neighborhoods, and to see if there consists any relationship between demographics and venues within a neighborhood. This will provide us with sufficient data to analyize and visualize each of the neighborhoods within Chicago and Houston.